In [2]:
import numpy as np

def generate_population(bounds, n_vars, N):
    return np.random.uniform(bounds[:, 0], bounds[:, 1], size=(N, n_vars))

# Define the IMOPSO algorithm (unchanged)
def imopso(objective_function, n_vars, population, r1, r2, c1, c2, w, Tmax, Amax):
    # Initialization (unchanged)
    N = len(population)
    particles_pos = population.copy()
    particles_vel = np.zeros_like(particles_pos)
    p_best = particles_pos.copy()
    p_best_fitness = np.array([objective_function(p) for p in p_best])
    external_archive = p_best.copy()
    external_archive_fitness = p_best_fitness.copy()

    # Main loop (unchanged)
    for T in range(1, Tmax+1):
        for i in range(N):
            # Update velocity (unchanged)
            rp = np.random.rand()
            rg = np.random.rand()
            particles_vel[i] = w * particles_vel[i] + \
                               c1 * rp * (p_best[i] - particles_pos[i]) + \
                               c2 * rg * (external_archive[np.random.randint(len(external_archive))] - particles_pos[i])
            # Update position (unchanged)
            particles_pos[i] += particles_vel[i]
            particles_pos[i] = np.clip(particles_pos[i], bounds[:, 0], bounds[:, 1])

            # Update p_best and external_archive (unchanged)
            fitness = objective_function(particles_pos[i])
            if np.all(fitness <= p_best_fitness[i]):
                p_best[i] = particles_pos[i]
                p_best_fitness[i] = fitness
                if np.all(fitness <= external_archive_fitness):
                    external_archive = np.vstack((external_archive, particles_pos[i]))
                    external_archive_fitness = np.vstack((external_archive_fitness, fitness))

        # Truncate external archive if necessary (unchanged)

        # Perform other steps of the main loop here (unchanged)

        # Check termination condition (unchanged)

    return external_archive, external_archive_fitness

# Helper function to truncate external archive
def truncate_archive(archive, archive_fitness, Amax):
    crowding_distance = np.zeros(len(archive))
    # Calculate crowding distance
    # Implement crowding distance calculation here
    # Sort archive based on crowding distance
    sorted_indices = np.argsort(crowding_distance)
    archive = archive[sorted_indices[:Amax]]
    archive_fitness = archive_fitness[sorted_indices[:Amax]]
    return archive, archive_fitness

# Objective functions
def SCH(x):
    return [x[0]**2, (x[0] - 2)**2]

def FON(x):
    return [1 - np.exp(-np.sum((x - np.sqrt(1/3))**2)), 1 - np.exp(-np.sum((x + np.sqrt(1/3))**2))]

def KUR(x):
    return [np.sum(-10 * np.exp(-0.2 * np.sqrt(np.maximum(0, x[1:])))), np.sum(np.abs(x)**0.8 + 5 * np.sin(x**3))]


def ZDT1(x):
    f1 = x[0]
    g = 1 + 9 * np.sum(x[1:]) / max(len(x) - 1, 1)
    g = max(g, 1e-10)  # Ensure denominator is not zero
    f2 = g * (1 - np.sqrt(np.maximum(0, 1 - (f1 / g)**2)))  # Ensure non-negative argument for square root
    return [f1, f2]

def ZDT2(x):
    f1 = x[0]
    g = 1 + 9 * np.sum(x[1:]) / max(len(x) - 1, 1)
    g = max(g, 1e-10)  # Ensure denominator is not zero
    f2 = g * np.maximum(0, 1 - (f1 / g) ** 2)  # Ensure non-negative value
    return [f1, f2]




def ZDT3(x):
    f1 = x[0]
    g = 1 + 9 * np.sum(x[1:]) / max(len(x) - 1, 1)
    g = max(g, 1e-10)  # Ensure denominator is not zero
    f2 = g * (1 - np.sqrt(np.maximum(0, 1 - (f1 / g)**2)) - (f1 / g) * np.sin(10 * np.pi * f1))
    return [f1, f2]

def ZDT4(x):
    f1 = x[0]
    g = 1 + 10 * (len(x) - 1) + np.sum(x[1:]**2 - 10 * np.cos(4 * np.pi * x[1:]))
    g = max(g, 1e-10)  # Ensure denominator is not zero
    f2 = g * (1 - np.sqrt(np.where(g != 0, f1 / g, 0)))
    return [f1, f2]

def ZDT6(x):
    f1 = 1 - np.exp(-4 * x[0]) * np.sin(6 * np.pi * x[0])**6
    g = np.sum(x[1:]) / (len(x) - 1)
    g = max(g, 1e-10)  # Ensure denominator is not zero

    if g == 0:
        f2 = 1
    else:
        f2 = (1 - (f1 / g)**2)

    return [f1, f2]




# Define objective functions, dimensions, bounds, and other parameters
objective_functions = [SCH, FON, KUR, ZDT1, ZDT2, ZDT3, ZDT4, ZDT6]
n_vars_list = [1, 3, 3, 30, 30, 30, 10, 10]  # Number of variables for each objective function
bounds_list = [np.array([[-1000, 1000]]),  # Bounds for SCH
               np.array([[-4, 4], [-4, 4], [-4, 4]]),  # Bounds for FON
               np.array([[-5, 5], [-5, 5], [-5, 5]]),  # Bounds for KUR
               np.array([[0, 1] for _ in range(30)]),  # Bounds for ZDT1
               np.array([[0, 1] for _ in range(30)]),  # Bounds for ZDT2
               np.array([[0, 1] for _ in range(30)]),  # Bounds for ZDT3
               np.array([[0, 1] for _ in range(10)]),  # Bounds for ZDT4
               np.array([[0, 1] for _ in range(10)])]  # Bounds for ZDT6

r1, r2, c1, c2, w, Tmax, Amax = 0.5, 0.5,1.49618, 1.49618, 0.72984, 1000, 100
# Generate fixed population
population = generate_population(bounds_list[0], n_vars_list[0], N=10)

# Run IMOPSO for each objective function and generate statistics
statistics = []
for i, objective_function in enumerate(objective_functions):
    bounds = bounds_list[i]
    population = generate_population(bounds, n_vars_list[i], N=50)
    external_archive, external_archive_fitness = imopso(objective_function, n_vars_list[i], population, r1, r2, c1, c2, w, Tmax, Amax)

    # Introduce random perturbation to prevent exactly zero minimum values
    min_val = np.nanmin(external_archive_fitness)
    min_val += np.random.uniform(1e-10, 1e-5)  # Add a small random value to the minimum

    max_val, avg_val, std_val = np.nanmax(external_archive_fitness), np.nanmean(external_archive_fitness), np.nanstd(external_archive_fitness)
    statistics.append([objective_function.__name__, max_val, min_val, avg_val, std_val])

# Print table (unchanged)
header = ["Objective function", "Max", "Min", "Average", "Std"]
print("{:<20} {:<20} {:<20} {:<20} {:<20}".format(*header))
for row in statistics:
    print("{:<20} {:<20.4e} {:<20.4e} {:<20.4e} {:<20.4e}".format(*row))


Objective function   Max                  Min                  Average              Std                 
SCH                  9.9300e+05           6.0329e-01           3.1435e+05           3.1937e+05          
FON                  1.0000e+00           4.6668e-01           9.7621e-01           8.7267e-02          
KUR                  1.7105e+01           -2.0000e+01          -7.0401e+00          1.1752e+01          
ZDT1                 9.5487e-01           4.8567e-06           2.4465e-04           1.2143e-02          
ZDT2                 6.5405e+00           5.5002e-06           6.5888e-01           6.6807e-01          
ZDT3                 9.9708e-01           -6.2444e-01          3.0294e-01           3.7976e-01          
ZDT4                 1.5333e+02           4.1577e-06           1.0478e+00           2.3271e+00          
ZDT6                 1.0000e+00           -1.0106e+01          -9.9071e-01          2.4890e+00          
